In [1]:
print ("helloworld")

helloworld


In [4]:
!pip install langchain langchain_community langchain_chroma langchain_openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 KB 12.4 MB/s eta 0:00:0000:01


In [4]:
!source ~/.bashrc

In [42]:
# 导入必要的库
import bs4
import os
os.environ["OPENAI_API_KEY"] = "sk-UtASYrjftCMXZhV36fC14c4b65394dF494640d2e5538E53e"
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI



In [31]:
import requests
import json

# 设置 API 密钥和基础 URL
base_url = "https://api.javis3000.com/v1"

# 设置请求头
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# 准备请求数据
data = {
    "input": "Hello, world!",
    "model": "text-embedding-3-large"
}

# 发送请求
try:
    response = requests.post(f"{base_url}/embeddings", headers=headers, json=data)
    
    # 检查响应状态
    if response.status_code == 200:
        print("连接成功！")
        result = response.json()
        print("嵌入向量的前 5 个元素:")
        print(result['data'][0]['embedding'][:5])
    else:
        print(f"错误: HTTP 状态码 {response.status_code}")
        print("响应内容:")
        print(json.dumps(response.json(), indent=2))
except requests.exceptions.RequestException as e:
    print(f"请求异常: {e}")

连接成功！
嵌入向量的前 5 个元素:
[-0.005830363, -0.024080755, -0.02201979, 0.02761384, -0.0027602217]


In [4]:
# 使用 WebBaseLoader 从网页加载内容，并仅保留标题、标题头和文章内容
bs4_strainer = bs4.SoupStrainer(class_=("h3-mktg lh-condensed mb-3 color-fg-default", "d-flex flex-items-center mb-6px", "d-flex flex-wrap flex-justify-center gutter-spacious"))
loader = WebBaseLoader(
    web_paths=("https://github.blog/ai-and-ml/generative-ai/what-is-retrieval-augmented-generation-and-what-does-it-do-for-generative-ai/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [5]:
# 检查加载的文档内容长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度

12210


In [6]:
# 查看第一个文档（前100字符）
print(docs[0].page_content[:500])

What is retrieval-augmented generation, and what does it do for generative AI?


Nicole Choi·@nicchoi29 




One of the hottest topics in AI right now is RAG, or retrieval-augmented generation, which is a retrieval method used by some AI tools to improve the quality and relevance of their outputs.
Organizations want AI tools that use RAG because it makes those tools aware of proprietary data without the effort and expense of custom model training. RAG also keeps models up to date.  When generati


In [7]:
# 使用 RecursiveCharacterTextSplitter 将文档分割成块，每块1000字符，重叠200字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [8]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量

18


In [9]:
print(len(all_splits[0].page_content))  # 打印第一个块的字符数

103


In [10]:
print(all_splits[0].page_content)  # 打印第一个块的内容

What is retrieval-augmented generation, and what does it do for generative AI?


Nicole Choi·@nicchoi29


In [11]:
print(all_splits[0].metadata)  # 打印第一个块的元数据

{'source': 'https://github.blog/ai-and-ml/generative-ai/what-is-retrieval-augmented-generation-and-what-does-it-do-for-generative-ai/', 'language': 'No language found.', 'start_index': 0}


In [12]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("请设置OPENAI_API_KEY环境变量")
embeddings = OpenAIEmbeddings(
    base_url="https://api.javis3000.com/v1",
    model="text-embedding-3-large",
    openai_api_key=openai_api_key
)
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

In [44]:
try:
    test_embedding = embeddings.embed_query("Hello, world!")
    print(f"测试嵌入的长度: {len(test_embedding)}")
    print(f"测试嵌入的前5个元素: {test_embedding[:5]}")
except Exception as e:
    print(f"嵌入测试失败: {e}")

测试嵌入的长度: 3072
测试嵌入的前5个元素: [-0.005830362904816866, -0.024080755189061165, -0.02201979048550129, 0.027613839134573936, -0.002760221716016531]


In [46]:
import langchain
print(f"LangChain 版本: {langchain.__version__}")

LangChain 版本: 0.2.16


In [13]:
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)

Add of existing embedding ID: 1


Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3


In [14]:
# 使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将分割的文档块嵌入并存储
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings()
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-UtASY***************************************E53e. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [53]:
from langchain_chroma import Chroma

try:
    vector_store = Chroma(
        collection_name="example_collection",
        embedding_function=embeddings,
        persist_directory="./chroma_langchain_db",
    )
    print("向量存储创建成功")
except Exception as e:
    print(f"创建向量存储失败: {e}")

向量存储创建成功


In [54]:
try:
    vector_store.add_texts(["这是第一个测试文档", "这是第二个测试文档"])
    print("成功添加测试文档到向量存储")
except Exception as e:
    print(f"添加测试文档失败: {e}")

成功添加测试文档到向量存储


In [55]:
try:
    results = vector_store.similarity_search("测试文档", k=1)
    print("搜索结果:", results)
except Exception as e:
    print(f"搜索失败: {e}")

搜索结果: [Document(page_content='这是第一个测试文档')]


In [15]:
if 'all_splits' in locals() and all_splits:
    try:
        vector_store.add_documents(all_splits)
        print(f"成功添加 {len(all_splits)} 个文档到向量存储")
    except Exception as e:
        print(f"添加文档失败: {e}")
else:
    print("all_splits 未定义或为空")

成功添加 18 个文档到向量存储


In [16]:
# 查看 vectorstore 数据类型
type(vector_store) 

langchain_chroma.vectorstores.Chroma

### Step 4: 检索文档

- **描述**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法，从向量数据库中检索与查询最相关的文档片段。
- **重要代码抽象**:
  - 类: `VectorStoreRetriever`
  - 方法: `as_retriever()`, `invoke()`
- **代码解释**:
  - **文档检索**: 将向量存储转换为检索器，并基于查询执行相似性搜索，获取相关文档片段。
  - **检查检索数量**: 打印检索到的文档片段数量，确保检索操作成功。
  - **验证检索内容**: 输出第一个检索到的文档内容，确认检索结果与预期相符。

在 LangChain 中，所有向量数据库都支持**vectorstore.as_retriever** 方法，实例化该数据库对应的检索器（Retriever），数据类型为`VectorStoreRetriever`，详细 API 文档：https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html

In [17]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [18]:
type(retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [45]:
retrieved_docs = retriever.invoke("什么是RAG?")

In [46]:
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量

6


In [47]:
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

RAG can also retrieve information from external and internal search engines. When integrated with an external search engine, RAG can search and retrieve information from the entire internet. When integrated with an internal search engine, it can also access information from within your organization, like an internal website or platform. Integrating both kinds of search engines supercharges RAG’s ability to provide relevant responses.


### Step 5: 生成回答

- **描述**: 将之前构建的组件（检索器、提示、LLM等）组合成一个完整的链条，实现用户问题的检索与生成回答。完整链条：输入用户问题，检索相关文档，构建提示，将其传递给模型（使用`ChatOpenAI` 类的 `invoke()` 方法），并解析输出生成最终回答。
- **重要代码抽象**:
  - 类: `ChatOpenAI`
  - 方法: `invoke()`
  - 类: `RunnablePassthrough`
  - 类: `StrOutputParser`
  - 模块：`hub`
- **代码解释**:
  - **模型初始化**: 使用 `ChatOpenAI` 类初始化一个 `GPT-4o-mini` 模型，准备处理生成任务。
  - **文档格式化**: 定义 `format_docs` 函数，用于将检索到的文档内容格式化为字符串。
  - **构建 RAG 链**: 使用 LCEL (LangChain Execution Layer) 的 `|` 操作符将各个组件连接成一个链条，包括文档检索、提示构建、模型调用以及输出解析。
  - **生成回答**: 使用 `stream()` 方法逐步输出生成的回答，并实时展示，确保生成的结果符合预期。

![retrieval](../images/retrieval.png)

#### LangChain Hub

`LangChain Hub` (https://smith.langchain.com/hub) 是一个提示词模板开源社区，为开发者提供了大量开箱即用的提示词模板。属于 `LangSmith` 产品的一部分。

下面我们尝试使用 RAG 应用的提示词模板：https://smith.langchain.com/hub/rlm/rag-prompt


```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

In [22]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI(model="gpt-4o")

In [23]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("rlm/rag-prompt")

/home/toy/.local/lib/python3.10/site-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/home/toy/.local/lib/python3.10/site-packages/langsmith/client.py:5519: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [24]:
# 打印模板
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [25]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [26]:
# 查看提示词
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


#### ⭐️**LCEL 在 RAG 中的应用**⭐️

##### **LCEL 概述**

LCEL 是 LangChain 中的一个重要概念，它提供了一种统一的接口，允许不同的组件（如 `retriever`, `prompt`, `llm` 等）可以通过统一的 `Runnable` 接口连接起来。每个 `Runnable` 组件都实现了相同的方法，如 `.invoke()`、`.stream()` 或 `.batch()`，这使得它们可以通过 `|` 操作符轻松连接。

##### **LCEL 中处理的组件**

- **Retriever**: 负责根据用户问题检索相关文档。
- **Prompt**: 根据检索到的文档构建提示，供模型生成回答。
- **LLM**: 接收提示并生成最终的回答。
- **StrOutputParser**: 解析 LLM 的输出，只提取字符串内容，供最终显示。

##### **LCEL 运作机制**

- **构建链条**: 通过 `|` 操作符，我们可以将多个 `Runnable` 组件连接成一个 `RunnableSequence`。LangChain 会自动将一些对象转换为 `Runnable`，如将 `format_docs` 转换为 `RunnableLambda`，将包含 `"context"` 和 `"question"` 键的字典转换为 `RunnableParallel`。

- **数据流动**: 用户输入的问题会在 `RunnableSequence` 中依次经过各个 `Runnable` 组件。首先，问题会通过 `retriever` 检索相关文档，然后通过 `format_docs` 将这些文档转换为字符串。`RunnablePassthrough` 则直接传递原始问题。最后，这些数据被传递给 `prompt` 来生成完整的提示，供 LLM 使用。

##### **LCEL 中的关键操作**

- **格式化文档**: `retriever | format_docs` 将问题传递给 `retriever` 生成文档对象，然后通过 `format_docs` 将这些文档格式化为字符串。
- **传递问题**: `RunnablePassthrough()` 直接传递原始问题，保持原样。
- **构建提示**: `{"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt` 构建完整的提示。
- **运行模型**: `prompt | llm | StrOutputParser()` 运行 LLM 生成回答，并解析输出。

#### 使用 LCEL 构建 RAG Chain

下面我们将 LCEL 的概念与代码实现结合起来，展示了如何通过一系列 `Runnable` 组件来实现完整的 RAG 流程。通过 LCEL，LangChain 提供了高度模块化和可扩展的开发方式，使复杂任务的实现变得更加简单和高效。


In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 确保使用正确的 API 基础 URL 和密钥
llm = ChatOpenAI(
    model="gpt-4o",  # 请确认这是正确的模型名称
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://api.javis3000.com/v1"
)

# 定义提示模板
prompt = ChatPromptTemplate.from_template(
    "Answer the following question based on the context: {context}\n\nQuestion: {question}"
)

# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 重新定义 rag_chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [29]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [79]:
# 流式生成回答
for chunk in rag_chain.stream("What is RAG，用中文回答?"):
    print(chunk, end="", flush=True)

RAG，即检索增强生成（retrieval-augmented generation），是一种用于一些人工智能工具的检索方法，用以提高其输出的质量和相关性。通过使用RAG，AI工具可以在无需进行定制模型训练的情况下访问专有数据，并保持模型的最新状态。例如，GitHub Copilot Chat在回答开发者问题时，会使用内部搜索引擎从索引文件中找到相关代码或文本片段，并通过语义搜索来检索最相关的内容，从而生成更加相关的回复。RAG不仅可以从外部搜索引擎检索信息，也可以从内部搜索引擎中获取企业内部的信息，极大地增强了AI模型的上下文理解和响应能力。

In [30]:
# 流式生成回答
for chunk in rag_chain.stream("RAG的重要性？"):
    print(chunk, end="", flush=True)

RAG（检索增强生成）的重要性在于它能够显著提高AI工具生成输出的质量和相关性。其关键优势包括：

1. **利用专有数据**：RAG使AI工具可以访问组织内部的专有数据，而无需进行昂贵且耗时的自定义模型训练。这使得模型能够更加了解特定组织的特殊需求和数据。

2. **保持模型的最新性**：传统模型只能基于训练时的数据生成答案，而RAG允许模型利用最新的私有数据库，提供更为准确和及时的信息。这对于快速变化的领域尤为重要。

3. **扩展数据源**：RAG不仅可以使用模型原有的训练数据，还能整合来自外部和内部搜索引擎的数据源，从而大大增强生成输出的全面性和准确性。

4. **减少定制化成本**：与需要调整模型权重的微调（fine-tuning）方法相比，RAG不需要对模型进行大量定制，这在节约时间和资源方面具有明显优势。

综合这些因素，RAG成为当前AI领域的热门话题，并受到各组织的广泛关注和采用。

In [31]:
# 流式生成回答
for chunk in rag_chain.stream("RAG有啥用？"):
    print(chunk, end="", flush=True)

RAG (Retrieval-Augmented Generation) 有多个重要用途：

1. **提高生成内容的质量和相关性**：RAG通过检索相关信息来增强AI工具的输出质量和相关性。

2. **节省定制模型训练的成本和精力**：RAG使AI工具能够利用专有数据，而无需进行昂贵和耗时的定制模型训练。

3. **保持模型的最新性**：没有RAG的情况下，模型只能基于训练时的已有数据生成答案。而通过RAG，模型能够利用私人数据库中的最新信息，从而生成更为准确和及时的响应。

4. **结合内部和外部搜索引擎**：RAG可以与内部和外部搜索引擎集成，从而在整个互联网和组织内部平台中检索信息，大大增强了RAG提供相关响应的能力。

5. **语义搜索和排名**：在具体应用中，例如GitHub Copilot Chat，RAG使用语义搜索来分析和排名文档内容，从而找到最相关的代码或文本片段，生成开发者所需的答案。

总之，RAG为定制AI模型提供了一种有效的方法，确保输出内容与组织知识、最佳实践和最新互联网信息保持一致。

# Homework
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝试提出3个相关问题，测试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在 LangChain Hub 上找一个可用的 RAG 提示词模板，测试对比两者的召回率和生成质量。

### 自定义 Prompt 的示例

In [27]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模板
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

从hub上导入新的模板

In [32]:
newprompt = hub.pull("pwoc517/more-crafted-rag-prompt")

/home/toy/.local/lib/python3.10/site-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [33]:
# 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(newprompt.invoke({"context": "filler context", "question": "filler question"}).text)

# Your role
You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer to the questioner's needs from the documents you are given.


# Instruction
Your task is to answer the question using the following pieces of retrieved context delimited by XML tags.

<retrieved context>
Retrieved Context:
filler context
</retrieved context>


# Constraint
1. Think deeply and multiple times about the user's question\nUser's question:\nfiller question\nYou must understand the intent of their question and provide the most appropriate answer.
- Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate response based on what you understand.
2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
3. Generate a concise, logical answer. When generating the 

In [34]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | newprompt
    | llm
    | StrOutputParser()
)

In [35]:
# 使用自定义 prompt 生成回答
custom_rag_chain.invoke("RAG与其他技术差别?")

'RAG（检索增强生成）与其他技术的主要区别在于它能够超越模型训练数据的限制，使用外部和内部数据源来生成输出。传统的生成式AI只能引用训练时期存在的信息，而RAG可以从整个互联网或组织内部的数据库中检索最新的信息，从而提供更及时和相关的响应。此外，RAG无需像微调那样调整模型权重，避免了高成本和复杂的定制过程。通过这种方式，RAG使得AI工具能够利用专有数据，而不需要进行昂贵的定制模型训练，同时保持模型的最新状态。'

In [38]:
python_prompt = ChatPromptTemplate.from_template(
    "你是一个Python专家。请根据以下功能需求,生成相应的Python代码实现:\n\n{requirement}\n\n请只输出代码,不要有任何解释。"
)

java_prompt = ChatPromptTemplate.from_template(
    "你是一个Java专家。请根据以下功能需求,生成相应的Java代码实现:\n\n{requirement}\n\n请只输出代码,不要有任何解释。"
)

In [39]:
python_chain = python_prompt | llm | StrOutputParser()
java_chain = java_prompt | llm | StrOutputParser()

In [40]:
multi_code_chain = RunnableParallel(
    requirement=RunnablePassthrough(),
    python_code=python_chain,
    java_code=java_chain
)

In [41]:
requirement = "实现一个简单的计算器,可以进行加减乘除四则运算"

result = multi_code_chain.invoke(requirement)

print("Python代码实现:")
print(result["python_code"])
print("\nJava代码实现:")
print(result["java_code"])

Python代码实现:
```python
def add(x, y):
    return x + y

def subtract(x, y):
    return x - y

def multiply(x, y):
    return x * y

def divide(x, y):
    if y == 0:
        return "Error! Division by zero."
    return x / y

def calculator():
    print("Select operation:")
    print("1. Add")
    print("2. Subtract")
    print("3. Multiply")
    print("4. Divide")

    choice = input("Enter choice(1/2/3/4): ")

    if choice in ['1', '2', '3', '4']:
        num1 = float(input("Enter first number: "))
        num2 = float(input("Enter second number: "))

        if choice == '1':
            print(f"{num1} + {num2} = {add(num1, num2)}")

        elif choice == '2':
            print(f"{num1} - {num2} = {subtract(num1, num2)}")

        elif choice == '3':
            print(f"{num1} * {num2} = {multiply(num1, num2)}")

        elif choice == '4':
            result = divide(num1, num2)
            if result == "Error! Division by zero.":
                print(result)
            else:
   